# Dummification of adjectives

In [ ]:
!pip install bertopic

In [4]:
from bertopic import BERTopic

In [4]:
import pandas as pd
from google.colab import drive

drive.mount('/content/gdrive')
df = pd.read_excel("/content/gdrive/MyDrive/Ironhack/Final_project/df_GPT_descr.xlsx")

Mounted at /content/gdrive


In [ ]:
# define new adj_df and drop all rows that dont have any adjectives
adj_df = df["descriptive_adjectives"].dropna().reset_index(drop=True)

In [ ]:
import ast

In [ ]:
# function that removes the "hidden" characteristic from the end of the descriptive adjectives column - need to apply to Series with .apply(lambda)
def clean_adj(text):
  string = text.replace(".","").replace("hidden:", "")\
  .replace("[","").replace("]","").replace("'","").replace("es: ","").replace("Ye","")\
  .replace("N","").lower()
  return string

def strip_strings(lst):
    return [s.strip() for s in lst]

# apply clean_adj to adj_df
adj_list = adj_df.apply(lambda x: clean_adj(x))

In [ ]:
# create new column in df with cleaned adjectives
df["descriptive_adjectives"] = df["descriptive_adjectives"].apply(lambda x: strip_strings(clean_adj(x).split()) if type(x) == str else "")

In [ ]:
len(df["descriptive_adjectives"].explode().unique())

4892

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

# Initialize MultiLabelBinarizer instance
mlb = MultiLabelBinarizer()

# Transform column with lists of words into columns for each word
dummies = pd.DataFrame(mlb.fit_transform(df["descriptive_adjectives"]), columns=mlb.classes_, index=df.index)

# Concatenate original DataFrame with the dummified columns
df = pd.concat([df, dummies], axis=1)

In [ ]:
df

In [ ]:
# show more content in the output of all cells
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
adj_topics, adj_probs = adj_model.fit_transform(adj_list)

In [ ]:
adj_model.get_topic_info()

# Make MVP happen

In [ ]:
# show more content in the output of all cells
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 100)
pd.set_option('display.max_colwidth', 100)

In [8]:
# only needed for model with NaNs in model_input removed
df.dropna(subset=["model_input"], inplace=True)
df.reset_index(inplace=True)

In [5]:
from joblib import dump, load
import pandas as pd
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [17]:
df =  pd.read_csv("/content/gdrive/MyDrive/Ironhack/Final_project/df_with_model_input.csv")

In [ ]:
### not using this as model is currently fitted to df without these 103 rows
# fill NaNs with names in "model_input" column
df["model_input"].fillna(df.loc[df["model_input"].isna()]["name"], inplace=True)

In [7]:
trial_model = BERTopic.load('/content/gdrive/MyDrive/Ironhack/Final_project/BERT_model_custom_embeddings_v1.joblib')

In [18]:
# assign topics to every item in the df
df["topic"] = trial_model.topics_

In [10]:
selection_df = pd.DataFrame(data=[df.loc[0,:]])

,level_0,index,Unnamed: 0,name,reference,geometry,formatted_address,price_level,rating,user_ratings_total,...,website,international_phone_number,photos,vicinity,url,business_status,permanently_closed,plus_code,model_input,topic
0,0,0,0,Restaurant „Zur S-Bahn“,ChIJAergcnhJqEcRST9W0VduzeY,"{'location': {'lat': 52.5125038, 'lng': 13.587...","Heinrich-Grüber-Straße 1, 12621 Berlin, Germany",2.0,4.6,1551.0,...,http://www.zur-s-bahn.de/,+49 30 5627003,"[{'height': 3584, 'html_attributions': ['<a hr...","Heinrich-Grüber-Straße 1, Berlin",https://www.google.com/maps/place/?q=place_id:...,OPERATIONAL,NaN,"{'compound_code': 'GH6Q+X4 Berlin', 'global_co...","'review0': 'authentic german cuisine, friendly...",0


In [12]:
user_input_name = pd.DataFrame(data=[df.loc[2980,:]])

In [19]:
def find_topic(model, user_input):
  if type(user_input) == str:
    print("str")
    input_topic, input_prob = model.transform(user_input)
    name = "None"
  elif type(user_input) != "None":
    print("df")
    input_topic, input_prob = model.transform(str(user_input["model_input"]))
    name = user_input["name"].to_string(index=False)
  else:
    print("error while assigning the topic for user input")
  return name, input_topic, input_prob 

In [20]:
import random
def output_recommendation(user_input_name, user_input_topic, user_input_prob, df):
  # remove user input from possible list of output if it's a restaurant
  if user_input_name == "None":
    return df.iloc[random.choice(range(len(df[df["topic"] == user_input_topic[0]])))]["name"]
  elif user_input_name != "None":
    df = df[~(df["name"].str.lower() == user_input_name.lower())]
    return df.iloc[random.choice(range(len(df[df["topic"] == user_input_topic[0]])))]["name"]
  else:
    return "error while selecting the recommendation"

In [91]:
df[df["name"] == "Ebert"]["model_input"]

22    'review0': 'this place has really good traditi...
Name: model_input, dtype: object

In [14]:
name, input_topic, input_prob = find_topic(trial_model, user_input_name)

df


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [21]:
output_recommendation(name, input_topic, input_prob, df)

'A-Petit Restaurant Berlin'

In [ ]:
trial_model.get_topic(0)

In [ ]:
trial_model.get_topic(input_topic[0])

In [9]:
trial_model.get_topic_info()

,Topic,Count,Name
0,-1,581,-1_service_great_place_delicious
1,0,735,0_pizza_italian_pasta_service
2,1,645,1_german_great_service_place
3,2,524,2_service_great_friendly_place
4,3,303,3_kebab_doner_doner kebab_meat
5,4,207,4_burger_burgers_fries_place
6,5,185,5_indian_chicken_paneer_ordered
7,6,155,6_sushi_order_fresh_delivery
8,7,122,7_falafel_shawarma_hummus_place
9,8,103,8_nan nan_nan_nan nan nan_
